# Sunbird RC API - Interactive Demo

This notebook demonstrates how to interact with Sunbird RC Registry API.

## Prerequisites
- Sunbird RC services running (use `./start-sunbird.sh`)
- Registry API available at http://localhost:8081

## Setup

In [1]:
import requests
import json
import pandas as pd
from datetime import datetime
from IPython.display import display, Markdown, JSON

# Configuration
BASE_URL = "http://localhost:8081/api/v1"
HEADERS = {"Content-Type": "application/json"}

print("✅ Setup complete!")
print(f"Registry API: {BASE_URL}")

✅ Setup complete!
Registry API: http://localhost:8081/api/v1


## 1. Check Registry Health

In [2]:
response = requests.get("http://localhost:8081/health")
health = response.json()

print(f"Status: {health['result']['healthy']}")
print(f"Service: {health['result']['name']}")
display(JSON(health, expanded=True))

Status: True
Service: sunbirdrc-registry-api


<IPython.core.display.JSON object>

## 2. List All Degree Certificates

In [3]:
response = requests.get(f"{BASE_URL}/DegreeCertificate", headers=HEADERS)
data = response.json()

print(f"Total Certificates: {data['totalCount']}\n")

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    # Select key columns
    columns = ['studentName', 'studentId', 'degree', 'major', 'university', 'graduationDate', 'gpa', 'osid']
    display(df[columns])
else:
    print("No certificates found")

Total Certificates: 5



,studentName,studentId,degree,major,university,graduationDate,gpa,osid
0,Alice Johnson,STU2024004,PhD,Physics,Caltech,2024-06-01,3.98,1-32bd571a-bfb3-4c51-920b-b084f96fcec5
1,Carol Martinez,STU2024006,Master,Data Science,Berkeley,2024-05-25,3.90,1-9227c26b-b8ef-405c-9e67-d2ad6bba6e76
2,John Smith,STU2024002,Master,Data Science,Stanford University,2024-06-20,4.00,1-fa8cf434-6962-46a2-b916-59af9bc50f77
3,Jane Doe,STU2024003,Bachelor,Computer Science,MIT,2024-05-15,4.00,1-b625f26c-098b-4da4-b07a-2ca252c0263a
4,Bob Williams,STU2024005,Bachelor,Mathematics,Harvard,2024-05-20,3.85,1-b0973be6-a211-4562-aea6-44020cd5aeb6


## 3. Create a New Degree Certificate

In [4]:
new_certificate = {
    "studentName": "Joy Ghosh",
    "studentId": "STU2024005",
    "degree": "Master",
    "major": "Computer Science",
    "university": "MIT",
    "graduationDate": "2024-05-15",
    "gpa": 4.00,
    "honors": "Cum Laude",
    "certificateNumber": "CERT-2024-CS-009"
}

response = requests.post(
    f"{BASE_URL}/DegreeCertificate",
    headers=HEADERS,
    json=new_certificate
)

if response.status_code == 200:
    result = response.json()
    print("✅ Certificate created successfully!")
    print(f"ID: {result['result']['DegreeCertificate']['osid']}")
    display(JSON(result, expanded=True))
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

✅ Certificate created successfully!
ID: 1-bed4340e-0a9c-497f-9bb3-06e3dc6ef807


<IPython.core.display.JSON object>

## 4. Get a Specific Certificate by ID

In [5]:
# Get the first certificate's ID
response = requests.get(f"{BASE_URL}/DegreeCertificate")
certificates = response.json()['data']

if certificates:
    cert_id = certificates[0]['osid']
    print(f"Fetching certificate: {cert_id}\n")
    response = requests.get(f"{BASE_URL}/DegreeCertificate/{cert_id}")
    certificate = response.json()
    display(JSON(certificate, expanded=True))

Fetching certificate: 1-bed4340e-0a9c-497f-9bb3-06e3dc6ef807



<IPython.core.display.JSON object>

## 5. Update a Certificate

In [5]:
# Get first certificate
response = requests.get(f"{BASE_URL}/DegreeCertificate")
certificates = response.json()['data']

if certificates:
    cert_id = certificates[0]['osid']
    
    # Update the certificate
    update_data = certificates[0].copy()
    update_data['gpa'] = 4.0  # Update GPA
    update_data['honors'] = 'Summa Cum Laude'
    
    response = requests.put(
        f"{BASE_URL}/DegreeCertificate/{cert_id}",
        headers=HEADERS,
        json=update_data
    )
    
    if response.status_code == 200:
        print("✅ Certificate updated successfully!")
        display(JSON(response.json(), expanded=True))
    else:
        print(f"❌ Error: {response.status_code}")
        print(response.text)
else:
    print("No certificates available to update")

✅ Certificate updated successfully!


<IPython.core.display.JSON object>

## 6. Search Certificates

In [6]:
search_query = {
    "filters": {
        "degree": {"eq": "Master"}
    }
}

response = requests.post(
    f"{BASE_URL}/DegreeCertificate/search",
    headers=HEADERS,
    json=search_query
)

if response.status_code == 200:
    results = response.json()
    print(f"Found {results['totalCount']} Master's degrees\n")
    
    if results['totalCount'] > 0:
        df = pd.DataFrame(results['data'])
        display(df[['studentName', 'degree', 'major', 'university', 'gpa']])
    else:
        print("No results found")
else:
    print(f"❌ Error: {response.status_code}")
    print(response.text)

Found 2 Master's degrees



,studentName,degree,major,university,gpa
0,Carol Martinez,Master,Data Science,Berkeley,3.9
1,John Smith,Master,Data Science,Stanford University,4.0


## 7. Statistics & Visualization

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

response = requests.get(f"{BASE_URL}/DegreeCertificate")
data = response.json()

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    
    # Statistics
    print("📊 Certificate Statistics\n")
    print(f"Total Certificates: {len(df)}")
    print(f"Average GPA: {df['gpa'].mean():.2f}")
    print(f"\nDegree Distribution:")
    print(df['degree'].value_counts())
    print(f"\nUniversities:")
    print(df['university'].value_counts())
    
    # Visualizations
    fig, axes = plt.subplots(1, 2, figsize=(14, 5))
    
    # Degree distribution
    df['degree'].value_counts().plot(kind='bar', ax=axes[0], color='skyblue')
    axes[0].set_title('Certificates by Degree Type')
    axes[0].set_xlabel('Degree')
    axes[0].set_ylabel('Count')
    
    # GPA distribution
    df['gpa'].plot(kind='hist', bins=10, ax=axes[1], color='lightgreen', edgecolor='black')
    axes[1].set_title('GPA Distribution')
    axes[1].set_xlabel('GPA')
    axes[1].set_ylabel('Frequency')
    
    plt.tight_layout()
    plt.show()
else:
    print("No data available for visualization")

## 8. Bulk Create Certificates

In [ ]:
# Sample bulk data
bulk_certificates = [
    {
        "studentName": "Alice Johnson",
        "studentId": "STU2024004",
        "degree": "PhD",
        "major": "Physics",
        "university": "Caltech",
        "graduationDate": "2024-06-01",
        "gpa": 3.98,
        "honors": "Summa Cum Laude",
        "certificateNumber": "CERT-2024-PHY-003"
    },
    {
        "studentName": "Bob Williams",
        "studentId": "STU2024005",
        "degree": "Bachelor",
        "major": "Mathematics",
        "university": "Harvard",
        "graduationDate": "2024-05-20",
        "gpa": 3.85,
        "honors": "Magna Cum Laude",
        "certificateNumber": "CERT-2024-MATH-004"
    },
    {
        "studentName": "Carol Martinez",
        "studentId": "STU2024006",
        "degree": "Master",
        "major": "Data Science",
        "university": "Berkeley",
        "graduationDate": "2024-05-25",
        "gpa": 3.90,
        "honors": "Cum Laude",
        "certificateNumber": "CERT-2024-DS-005"
    }
]

print(f"Creating {len(bulk_certificates)} certificates...\n")

results = []
for i, cert in enumerate(bulk_certificates, 1):
    response = requests.post(
        f"{BASE_URL}/DegreeCertificate",
        headers=HEADERS,
        json=cert
    )
    
    if response.status_code == 200:
        result = response.json()
        osid = result['result']['DegreeCertificate']['osid']
        print(f"✅ {i}. Created: {cert['studentName']} (ID: {osid})")
        results.append({'status': 'success', 'osid': osid, 'name': cert['studentName']})
    else:
        print(f"❌ {i}. Failed: {cert['studentName']} - {response.status_code}")
        results.append({'status': 'failed', 'name': cert['studentName'], 'error': response.text})

print(f"\n✅ Created {len([r for r in results if r['status'] == 'success'])} certificates")
print(f"❌ Failed {len([r for r in results if r['status'] == 'failed'])} certificates")

## 9. Delete a Certificate

In [ ]:
# WARNING: This will delete a certificate!
# Uncomment to use

# cert_id_to_delete = "YOUR-CERTIFICATE-ID-HERE"
# response = requests.delete(f"{BASE_URL}/DegreeCertificate/{cert_id_to_delete}")

# if response.status_code == 200:
#     print(f"✅ Certificate {cert_id_to_delete} deleted successfully")
# else:
#     print(f"❌ Error: {response.status_code}")
#     print(response.text)

print("⚠️  Delete function is commented out for safety")
print("Uncomment the code above to enable deletion")

## 10. Export to CSV

In [ ]:
response = requests.get(f"{BASE_URL}/DegreeCertificate")
data = response.json()

if data['totalCount'] > 0:
    df = pd.DataFrame(data['data'])
    
    # Select columns to export
    export_columns = [
        'studentName', 'studentId', 'degree', 'major', 
        'university', 'graduationDate', 'gpa', 'honors', 
        'certificateNumber', 'osid', 'osCreatedAt'
    ]
    
    filename = f'degree_certificates_{datetime.now().strftime("%Y%m%d_%H%M%S")}.csv'
    df[export_columns].to_csv(filename, index=False)
    
    print(f"✅ Exported {len(df)} certificates to: {filename}")
    display(df[export_columns].head())
else:
    print("No certificates to export")

## Summary

This notebook demonstrated:
- ✅ Checking registry health
- ✅ Listing all certificates
- ✅ Creating new certificates
- ✅ Reading specific certificates
- ✅ Updating certificates
- ✅ Searching certificates
- ✅ Visualizing data
- ✅ Bulk operations
- ✅ Exporting to CSV

### Next Steps
- Explore other entity types (Issuer, RevokedCredential)
- Implement authentication
- Add credential signing/verification
- Create custom workflows

For more information, see [SETUP_GUIDE.md](./SETUP_GUIDE.md)